# Automação de Indicadores

### Objetivo: Treinar e criar um Projeto Completo que envolva a automatização de um processo feito no computador

### Descrição:

Imagine que você trabalha em uma grande rede de lojas de roupa com 25 lojas espalhadas por todo o Brasil.

Todo dia, pela manhã, a equipe de análise de dados calcula os chamados One Pages e envia para o gerente de cada loja o OnePage da sua loja, bem como todas as informações usadas no cálculo dos indicadores.

Um One Page é um resumo muito simples e direto ao ponto, usado pela equipe de gerência de loja para saber os principais indicadores de cada loja e permitir em 1 página (daí o nome OnePage) tanto a comparação entre diferentes lojas, quanto quais indicadores aquela loja conseguiu cumprir naquele dia ou não.

O papel, como Analista de Dados, é conseguir criar um processo da forma mais automática possível para calcular o OnePage de cada loja e enviar um email para o gerente de cada loja com o seu OnePage no corpo do e-mail e também o arquivo completo com os dados da sua respectiva loja em anexo.

### Passo 1 - Importar Arquivos e Bibliotecas

In [2]:
import pandas as pd
import pathlib
import win32com.client as win32

In [3]:
vendas_df = pd.read_excel(r'E:\Downloads\Projeto AutomacaoIndicadores\Bases de Dados\Vendas.xlsx')
emails_df = pd.read_excel(r'E:\Downloads\Projeto AutomacaoIndicadores\Bases de Dados\Emails.xlsx')
lojas_df = pd.read_csv(r'E:\Downloads\Projeto AutomacaoIndicadores\Bases de Dados\Lojas.csv', encoding='latin-1', sep=';')

# jogando o nome das lojas para o vendas_df
vendas_df = vendas_df.merge(lojas_df, on='ID Loja')

In [4]:
display(vendas_df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [ ]:
# percorrendo cada loja no df lojas_df e criando um dicionário para cada uma
tab_lojas = {}
for loja in lojas_df['Loja']:
   tab_lojas[loja] = vendas_df.loc[vendas_df['Loja']==loja, :]

display(tab_lojas)

In [25]:
dia_indicador = vendas_df['Data'].max()

print(dia_indicador)

2019-12-26 00:00:00


### Passo 3 - Salvar a planilha na pasta de backup

In [7]:
# pegando o caminho da pasta backup
caminho_backup = pathlib.Path(r'E:\Downloads\Projeto AutomacaoIndicadores\Backup Arquivos Lojas')

# adiciona os arquivos da pasta do backup em uma variável
arquivos_backup = caminho_backup.iterdir()

# adicionando os nomes dos arquivos em uma lista
lista_arquivos = []
for arquivo in arquivos_backup:
    lista_arquivos.append(arquivo.name)

# percorrendo a lista verificando se ja existe o arquivo e criando caso n tenha
for loja in lojas_df['Loja']:
    if loja not in lista_arquivos:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

# criando e formatando o nome do arquivo de cada loja, dentro do caminho especificado 
    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo

# salvando os df no local desejado
    tab_lojas[loja].to_excel(local_arquivo)

### Passo 4 - Calcular o indicador para 1 loja

In [8]:
# escolhendo uma loja para testar o código
loja = 'Iguatemi Esplanada'

# pegando apenas o df da loja escolhida (vendas do ano)
tab_lojas_ano = tab_lojas[loja]

# separando as vendas do dia
tab_lojas_dia = tab_lojas_ano.loc[tab_lojas[loja]['Data'] == dia_indicador, :]

# fazendo o faturamento
faturamento_ano = tab_lojas_ano['Valor Final'].sum()
faturamento_dia = tab_lojas_dia['Valor Final'].sum()

In [ ]:
display(tab_lojas)

In [10]:
# tirando as duplicadas da coluna produtos e lendo quantos tem
div_produtos_ano = len(tab_lojas_ano['Produto'].unique())
div_produtos_dia = len(tab_lojas_dia['Produto'].unique())

display(div_produtos_dia)

3

In [11]:
# tirando as duplicadas da coluna código venda e somando a coluna valor final
cod_vendas_ano = tab_lojas_ano.groupby(['Código Venda']).sum()
cod_vendas_dia = tab_lojas_dia.groupby(['Código Venda']).sum()

# calculando o ticket médio (média dos valores por venda)
ticket_ano = cod_vendas_ano['Valor Final'].mean()
ticket_dia = cod_vendas_dia['Valor Final'].mean()

C:\Users\Léo\AppData\Local\Temp\ipykernel_9364\2880560444.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cod_vendas_ano = tab_lojas_ano.groupby(['Código Venda']).sum()
C:\Users\Léo\AppData\Local\Temp\ipykernel_9364\2880560444.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cod_vendas_dia = tab_lojas_dia.groupby(['Código Venda']).sum()


In [12]:
# dando entrada nas metas
meta_fat_dia = 1000
meta_fat_ano = 1650000
meta_qprod_dia = 4
meta_qprod_ano = 120
meta_ticket_dia = 500
meta_ticket_ano = 500

### Passo 5 - Enviar por e-mail para o gerente

In [13]:
# pegando o email da loja desejada
email_loja = emails_df.loc[emails_df['Loja']==loja, 'E-mail'].values[0]


print(email_loja)

ronaldo5331+Helena@hotmail.com


In [14]:
display(emails_df)

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,ronaldo5331+Helena@hotmail.com
1,Shopping Midway Mall,Alice,ronaldo5331+Alice@hotmail.com
2,Norte Shopping,Laura,ronaldo5331+Laura@hotmail.com
3,Shopping Iguatemi Fortaleza,Manuela,ronaldo5331+Manuela@hotmail.com
4,Shopping União de Osasco,Valentina,ronaldo5331+Valentina@hotmail.com
5,Shopping Center Interlagos,Sophia,ronaldo5331+Sophia@hotmail.com
6,Rio Mar Recife,Isabella,ronaldo5331+@hotmail.com
7,Salvador Shopping,Heloisa,ronaldo5331@hotmail.com
8,Rio Mar Shopping Fortaleza,Luiza,ronaldo5331@hotmail.com
9,Shopping Center Leste Aricanduva,Julia,ronaldo5331@hotmail.com


In [ ]:
# enviando o email através do app outlook no pc
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreatItem(0)
mail.To = emails_df.loc[emails_df['Loja']==loja, 'E-mail'].values[0]
nome = emails_df.loc[emails_df['Loja']==loja, 'Gerente'].values[0]
mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
mail.Body = 'conteúdo'
attachment = pathlib.Path(r'{}/{}/{}_{}_{}.xlsx'.format(caminho_backup, loja, dia_indicador.day, dia_indicador.month, loja))
mail.Attachment.add(str(attachment))

mail.Send()

### Passo 6 - Automatizar todas as lojas

In [ ]:
# passando por todas as lojas
for loja in tab_lojas:
    # pegando apenas o df da loja escolhida (vendas do ano)
    tab_lojas_ano = tab_lojas[loja]

    # separando as vendas do dia
    tab_lojas_dia = tab_lojas_ano.loc[tab_lojas[loja]['Data'] == dia_indicador, :]

    # fazendo o faturamento
    faturamento_loja_ano = tab_lojas_ano['Valor Final'].sum()
    faturamento_loja_dia = tab_lojas_dia['Valor Final'].sum()

    # tirando as duplicadas da coluna produtos e lendo quantos tem
    div_produtos_ano = len(tab_lojas_ano['Produto'].unique())
    div_produtos_dia = len(tab_lojas_dia['Produto'].unique())

    # tirando as duplicadas da coluna código venda e somando a coluna valor final
    cod_vendas_ano = tab_lojas_ano.groupby(['Código Venda']).sum()
    cod_vendas_dia = tab_lojas_dia.groupby(['Código Venda']).sum()

    # calculando o ticket médio (média dos valores por venda)
    ticket_ano = cod_vendas_ano['Valor Final'].mean()
    ticket_dia = cod_vendas_dia['Valor Final'].mean()

    # enviando o email através do app outlook no pc
    outlook = win32.Dispatch('outlook.application')

    mail = outlook.CreatItem(0)
    mail.To = emails_df.loc[emails_df['Loja']==loja, 'E-mail'].values[0]
    nome = emails_df.loc[emails_df['Loja']==loja, 'Gerente'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
    mail.Body = 'conteúdo'
    attachment = pathlib.Path(r'{}/{}/{}_{}_{}.xlsx'.format(caminho_backup, loja, dia_indicador.day, dia_indicador.month, loja))
    mail.Attachment.add(str(attachment))

    mail.Send()


### Passo 7 - Criar ranking para diretoria

In [28]:
# ranking por meio do faturamento, somando as infos por cada loja, apresentando duas colunas e colocando em ordem decrescente
faturamento_ano_df = vendas_df.groupby('Loja')[['Loja', 'Valor Final']].sum().sort_values('Valor Final', ascending=False)

# ranking diário
faturamento_dia_df = vendas_df.loc[vendas_df['Data']==dia_indicador, :]
faturamento_dia_df = faturamento_dia_df.groupby('Loja')[['Loja', 'Valor Final']].sum().sort_values('Valor Final', ascending=False)
display(faturamento_dia_df)

C:\Users\Léo\AppData\Local\Temp\ipykernel_9364\1146999644.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  faturamento_ano_df = vendas_df.groupby('Loja')[['Loja', 'Valor Final']].sum().sort_values('Valor Final', ascending=False)
C:\Users\Léo\AppData\Local\Temp\ipykernel_9364\1146999644.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  faturamento_dia_df = faturamento_dia_df.groupby('Loja')[['Loja', 'Valor Final']].sum().sort_values('Valor Final', ascending=False)


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937
